## Accessing ECMWF Open Data – Real Time

The Planetary Computer includes data from the ECMWF Open Data (real time) program. See the [dataset](https://planetarycomputer.microsoft.com/dataset/ecmwf-forecast) page and [ECMWF Uesr Guide](https://confluence.ecmwf.int/display/UDOC/ECMWF+Open+Data+-+Real+Time) for more.

A separate collection exists for each combination of *stream* (or forecasting system) and *type*, namely, enfo-ep, oper-fc, scda-fc, scwv-fc, waef-ef, waef-ep, wave-fc. This notebook will focus on the wave-fc collection.

Each item in this collection includes metadata about the data that produced that particular dataset. Filter on these values to select the item of interest. For example, we can select data that are forecast `0h` hours out.

In [1]:
import cartopy.crs as ccrs
import fsspec
import matplotlib.pyplot as plt
import pystac_client
import planetary_computer
import urllib.request
import xarray as xr

#Loading custom codec
from stactools.ecmwf_forecast.range_codec import Range
from numcodecs.registry import register_codec
register_codec(Range)

/anaconda/envs/py38_default/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


In [ ]:
catalog = pystac_client.Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["ecmwf-forecast-wave-fc"],
    query={
        "ecmwf:step": {"eq": "0h"},
    },
)
items = search.get_all_items()
len(items)

/anaconda/envs/py38_default/lib/python3.8/site-packages/pystac_client/item_search.py:846: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


We'll select the most recent item, using the item's datetime.

In [ ]:
item = max(items, key=lambda item: item.datetime)
item

This STAC item has two assets. One asset is the actual GRIB2 file with the data. The second asset is the "index" file, which contains information about the messages within the GRIB2 file.

In [ ]:
url = item.assets["data"].href
url

To open the file with xarray, we can download it locally and open it with `cfgrib`.

In [ ]:
filename, _ = urllib.request.urlretrieve(url)
ds = xr.open_dataset(filename, engine="cfgrib")
ds

As opposed to having to download the data locally to take a look at it, we can open it more efficiently using kerchunk indices. [Kerchunk](https://pypi.org/project/kerchunk/) is a python library that provides a unified way to represent a variety of chunked, compressed data formats (e.g. NetCDF, HDF5, GRIB) through metadata, allowing efficient access to the data from traditional file systems or cloud object storage.

We used the Kerchunk library to generate metadata indices for this STAC collection and stored them in the properties section for each item. Let's take a look.

In [ ]:
item.properties['kerchunk_indices']

Kerchunk indices are similar to a Zarr store. We can use the packages fsspec and xarray to view the virtual dataset. You'll see it matches the dataset loaded above from the downloaded .grib2 file.

In [ ]:
m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'])
ds_kerchunk = xr.open_dataset(m, engine='zarr', consolidated=False)
ds_kerchunk

We can plot the various data variables, for example the significant height of combined wind waves and swell.

In [ ]:
projection = ccrs.Robinson()
fig, ax = plt.subplots(figsize=(16, 9), subplot_kw=dict(projection=projection))

ds.swh.plot(ax=ax)
plt.show()

Now let's do a simple comparison of loading and analysis time between having to retrieve/download the data and accessing it via kerchunk.

Performing the analysis using the original .grib2 file takes 3 seconds (2.2 seconds to load, <1 second to calculate the mean of swh). Performing the analysis using kerchunk indices takes 1 second (<1 second to load, <1 second to calculate the mean of swh). When performing a more complex analysis, using the kerchunk indices can really speed up your work.

In [ ]:
%%time
filename, _ = urllib.request.urlretrieve(url)
ds = xr.open_dataset(filename, engine="cfgrib")

In [ ]:
%%time
ds['swh'].mean()

In [ ]:
%%time
m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'])
ds = xr.open_dataset(m, engine='zarr', consolidated=False)

In [ ]:
%%time
ds['swh'].mean()